## Answer single Questions

In [34]:
import xlnet
from utils import *

testmodel = xlnet.XLNET()

Didn't find file ./test-squad-trained_xlnet\spiece.model. We won't load it.
Didn't find file ./test-squad-trained_xlnet\added_tokens.json. We won't load it.
loading file None
loading file ./test-squad-trained_xlnet\tokenizer.json
loading file None
loading file ./test-squad-trained_xlnet\special_tokens_map.json
loading file ./test-squad-trained_xlnet\tokenizer_config.json
loading configuration file ./test-squad-trained_xlnet\config.json
Model config XLNetConfig {
  "_name_or_path": "./test-squad-trained_xlnet",
  "architectures": [
    "XLNetForQuestionAnsweringSimple"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "

In [35]:
testmodel.predict("who am I?", "My name is Roman.")

{'score': 0.9578208327293396, 'start': 0, 'end': 0, 'answer': ''}

In [36]:
testmodel.predict("who am I?", "My name is Roman but not Vincenzo.")

{'score': 1.0, 'start': 0, 'end': 0, 'answer': ''}

In [37]:
testmodel.predict("who am I?", "My name is Roman and Vincenzo.")

{'score': 0.9999890923500061, 'start': 0, 'end': 0, 'answer': ''}

In [38]:
testmodel.predict("who am I?", "My name is not Roman and but Vincenzo.")

{'score': 1.0, 'start': 0, 'end': 0, 'answer': ''}

## Answer multiple Questions

In [1]:
from transformers import Trainer, default_data_collator, AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,XLNetTokenizer
from datasets import load_dataset, load_metric
import torch
import numpy as np
from utils import *

In [2]:
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = AutoModelForQuestionAnswering.from_pretrained("./test-squad-trained_xlnet").to("cuda:0")

config = load_yaml_file('xlnet_config.yaml')

##
GPU_USAGE = config['GPU_USAGE']
BATCH_SIZE = config['BATCH_SIZE']
MAX_LENGTH = config['MAX_LENGTH']
DOC_STRIDE = config['DOC_STRIDE']
N_BEST_SIZE = config['N_BEST_SIZE']
MAX_ANSWER_LENGTH = config['MAX_ANSWER_LENGTH']
squad_v2 = config['squad_v2']
##

PAD_RIGHT = tokenizer.padding_side == "right"

datasets = load_dataset("squad_v2")
data_collator = default_data_collator

Reusing dataset squad_v2 (C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 117.65it/s]


In [3]:
torch.cuda.is_available()

True

In [4]:
tokenized_datasets = datasets.map(
    prepare_train_features, 
    fn_kwargs={
        'tokenizer':tokenizer, 
        'PAD_RIGHT':PAD_RIGHT,
        'MAX_LENGTH':MAX_LENGTH, 
        'DOC_STRIDE':DOC_STRIDE
        }, 
    batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-f7f821988ca865d4.arrow
Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-c9925728a42415f7.arrow


In [5]:
model_name = "test-squad-trained"

import os
os.environ["WANDB_DISABLED"] = "true"

args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,

)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [6]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    fn_kwargs={
        'tokenizer':tokenizer, 
        'PAD_RIGHT':PAD_RIGHT, 
        'MAX_LENGTH':MAX_LENGTH, 
        'DOC_STRIDE':DOC_STRIDE
        },
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-b8ab0739473eb6a6.arrow


In [7]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `XLNetForQuestionAnsweringSimple.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 12852
  Batch size = 8
100%|█████████▉| 1606/1607 [04:58<00:00,  7.08it/s]

In [8]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits', 'mems'])

In [9]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -N_BEST_SIZE - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -N_BEST_SIZE - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
                start_index >= len(offset_mapping)
                or end_index >= len(offset_mapping)
                or offset_mapping[start_index] is None
                or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > MAX_ANSWER_LENGTH:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:N_BEST_SIZE  ]
valid_answers

[{'score': 4.275517, 'text': 'France'},
 {'score': -3.3379405, 'text': 'France.'},
 {'score': -4.914199, 'text': 'a region in France'},
 {'score': -5.3332705, 'text': 'region in France'},
 {'score': -5.577037, 'text': 'Normandy, a region in France'},
 {'score': -6.4357605, 'text': 'in France'},
 {'score': -7.476758, 'text': ', a region in France'},
 {'score': -8.758042,
  'text': '10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -9.317292,
  'text': 'the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -9.40016,
  'text': 'in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -9.456315,
  'text': 'Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -10.007275,
  'text': 'gave their name to Normandy, a region in France'},
 {'score': -10.153693,
  'text': 'Latin: Normanni) were the people who in the 10th an

In [10]:
final_predictions = postprocess_qa_predictions(
    datasets["validation"], 
    validation_features, 
    raw_predictions.predictions,
    tokenizer, 
    config['squad_v2'], 
    n_best_size=config['N_BEST_SIZE'], 
    max_answer_length=config['MAX_ANSWER_LENGTH'])

Post-processing 11873 example predictions split into 12852 features.


100%|██████████| 11873/11873 [00:23<00:00, 501.37it/s]


In [11]:
metric = load_metric("squad_v2" if squad_v2 else "squad")
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 55.50408489850922,
 'f1': 56.9753646445723,
 'total': 11873,
 'HasAns_exact': 17.07152496626181,
 'HasAns_f1': 20.018303715419414,
 'HasAns_total': 5928,
 'NoAns_exact': 93.82674516400337,
 'NoAns_f1': 93.82674516400337,
 'NoAns_total': 5945,
 'best_exact': 55.50408489850922,
 'best_exact_thresh': 0.0,
 'best_f1': 56.975364644572494,
 'best_f1_thresh': 0.0}

In [42]:
bleus = calculate_bleu_score(formatted_predictions, references)
bleu_avg = np.sum(bleus)/len(bleus)
print('BLEU-2 avg:', bleu_avg, 'of', len(bleus), 'comparisons')

BLEU-2 avg: 0.8066689986769077 of 1424 comparisons


In [44]:
bleus = calculate_bleu_score_new(formatted_predictions, references)
bleu_avg = np.sum(bleus)/len(bleus)
print('BLEU-2 avg:', bleu_avg, 'of', len(bleus), 'comparisons')

BLEU-2 avg: 0.566554085245171 of 11873 comparisons


In [16]:
tp, fp, tn, fn = [], [], [], []
for i in range(len(formatted_predictions)):
    pred = formatted_predictions[i]['prediction_text']
    ref = references[i]['answers']['text']
    if (len(pred) == 0) and (len(ref) == 0):
        tn.append(i)
    elif (len(pred) != 0) and (len(ref) == 0):
        fp.append(i)
    elif (len(pred) == 0) and (len(ref) != 0):
        fn.append(i)
    elif (len(pred) != 0) and (len(ref) != 0):
        tp.append(i)
    
print('{} tp | {} fp\n-----------------\n{} fn | {} tn'.format(len(tp),len(fp),len(fn),len(tn)))

1424 tp | 367 fp
-----------------
4504 fn | 5578 tn


In [17]:
fp_n = 3
np.random.seed(42)
idxs_fp = np.random.choice(len(fp),fp_n,False)

for idx_fp in idxs_fp:
    idx_fp = fp[int(idx_fp)]
    print(str(idx_fp))
    print(str(datasets['validation'][idx_fp]))
    print('\n'+str(formatted_predictions[idx_fp]))
    print('\n'+str(references[idx_fp])+'\n\n\n')

5873
{'id': '5a25bd5cef59cd001a623cc6', 'title': 'Construction', 'context': "New techniques of building construction are being researched, made possible by advances in 3D printing technology. In a form of additive building construction, similar to the additive manufacturing techniques for manufactured parts, building printing is making it possible to flexibly construct small commercial buildings and private habitations in around 20 hours, with built-in plumbing and electrical facilities, in one continuous build, using large 3D printers. Working versions of 3D-printing building technology are already printing 2 metres (6 ft 7 in) of building material per hour as of January 2013[update], with the next-generation printers capable of 3.5 metres (11 ft) per hour, sufficient to complete a building in a week. Dutch architect Janjaap Ruijssenaars's performative architecture 3D-printed building is scheduled to be built in 2014.", 'question': 'What is flexible construction similar to?', 'answers

In [18]:
fn_n = 3
np.random.seed(42)
idxs_fn = np.random.choice(len(fn),fn_n,False)

for idx_fn in idxs_fn:
    idx_fn = fn[int(idx_fn)]
    print(str(idx_fn))
    print(str(datasets['validation'][idx_fn]))
    print('\n'+str(formatted_predictions[idx_fn]))
    print('\n'+str(references[idx_fn])+'\n\n\n')

5925
{'id': '572750e8dd62a815002e9af4', 'title': 'Construction', 'context': 'The project must adhere to zoning and building code requirements. Constructing a project that fails to adhere to codes does not benefit the owner. Some legal requirements come from malum in se considerations, or the desire to prevent things that are indisputably bad – bridge collapses or explosions. Other legal requirements come from malum prohibitum considerations, or things that are a matter of custom or expectation, such as isolating businesses to a business district and residences to a residential district. An attorney may seek changes or exemptions in the law that governs the land where the building will be built, either by arguing that a rule is inapplicable (the bridge design will not cause a collapse), or that the custom is no longer needed (acceptance of live-work spaces has grown in the community).', 'question': 'Who may seek changes or exemptions in the law that governs the land where the building w

In [19]:
from transformers import pipeline
model_cpu = AutoModelForQuestionAnswering.from_pretrained("test-squad-trained_xlnet")
qa_prediction = pipeline('question-answering', model=model_cpu, tokenizer=tokenizer)

loading configuration file test-squad-trained_xlnet\config.json
Model config XLNetConfig {
  "_name_or_path": "test-squad-trained_xlnet",
  "architectures": [
    "XLNetForQuestionAnsweringSimple"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "untie_r": true,
  "use_mems_eval": true,
  "use_mems_train": fal